<a href="https://colab.research.google.com/github/supriya-301iit/supriya-301iit/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Q1


1. Bollinger Bands

In [2]:
def bollinger_bands(df, window=20, num_std=2):
    df['MA'] = df['Close'].rolling(window).mean()
    df['BB_Upper'] = df['MA'] + num_std * df['Close'].rolling(window).std()
    df['BB_Lower'] = df['MA'] - num_std * df['Close'].rolling(window).std()
    return df


2.MACD

In [1]:
def macd(df, fast=12, slow=26, signal=9):
    df['EMA_fast'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df


3.Stochastic Oscillator

In [3]:
def stochastic_oscillator(df, k_period=14, d_period=3):
    low_min = df['Low'].rolling(window=k_period).min()
    high_max = df['High'].rolling(window=k_period).max()
    df['%K'] = 100 * ((df['Close'] - low_min) / (high_max - low_min))
    df['%D'] = df['%K'].rolling(window=d_period).mean()
    return df


4. VWAP

In [4]:
def vwap(df):
    cum_vol_price = (df['Close'] * df['Volume']).cumsum()
    cum_volume = df['Volume'].cumsum()
    df['VWAP'] = cum_vol_price / cum_volume
    return df


5.RSI

In [5]:
def rsi(df, period=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


6.ATR

In [6]:
def atr(df, period=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift())
    df['L-PC'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=period).mean()
    return df


#Q2

1. Bollinger Bands

In [7]:
def signal_bollinger_bands(df):
    buy = (df['Close'] < df['BB_Lower'])
    sell = (df['Close'] > df['BB_Upper'])

    df['BB_Signal'] = 0
    df.loc[buy, 'BB_Signal'] = 1
    df.loc[sell, 'BB_Signal'] = -1

    print("Bollinger Bands - Buy Signals:", buy.sum(), "Sell Signals:", sell.sum())
    return df


2.MACD

In [8]:
def signal_macd(df):
    buy = (df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift() <= df['MACD_Signal'].shift())
    sell = (df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift() >= df['MACD_Signal'].shift())

    df['MACD_Signal_Flag'] = 0
    df.loc[buy, 'MACD_Signal_Flag'] = 1
    df.loc[sell, 'MACD_Signal_Flag'] = -1

    print("MACD - Buy Signals:", buy.sum(), "Sell Signals:", sell.sum())
    return df


3.Stochastic Oscillator

In [9]:
def signal_stochastic(df):
    buy = (df['%K'] < 20) & (df['%D'] < 20) & (df['%K'] > df['%D'])
    sell = (df['%K'] > 80) & (df['%D'] > 80) & (df['%K'] < df['%D'])

    df['Stoch_Signal'] = 0
    df.loc[buy, 'Stoch_Signal'] = 1
    df.loc[sell, 'Stoch_Signal'] = -1

    print("Stochastic Oscillator - Buy Signals:", buy.sum(), "Sell Signals:", sell.sum())
    return df


4. VWAP

In [10]:
def signal_vwap(df):
    buy = (df['Close'] > df['VWAP']) & (df['Close'].shift() <= df['VWAP'].shift())
    sell = (df['Close'] < df['VWAP']) & (df['Close'].shift() >= df['VWAP'].shift())

    df['VWAP_Signal'] = 0
    df.loc[buy, 'VWAP_Signal'] = 1
    df.loc[sell, 'VWAP_Signal'] = -1

    print("VWAP - Buy Signals:", buy.sum(), "Sell Signals:", sell.sum())
    return df


5.RSI

In [11]:
def signal_rsi(df):
    buy = df['RSI'] < 30
    sell = df['RSI'] > 70

    df['RSI_Signal'] = 0
    df.loc[buy, 'RSI_Signal'] = 1
    df.loc[sell, 'RSI_Signal'] = -1

    print("RSI - Buy Signals:", buy.sum(), "Sell Signals:", sell.sum())
    return df


6.ATR

In [12]:
def signal_atr(df, multiplier=1.5):
    df['ATR_Upper'] = df['Close'].shift(1) + multiplier * df['ATR']
    df['ATR_Lower'] = df['Close'].shift(1) - multiplier * df['ATR']

    buy = df['Close'] > df['ATR_Upper']
    sell = df['Close'] < df['ATR_Lower']

    df['ATR_Signal'] = 0
    df.loc[buy, 'ATR_Signal'] = 1
    df.loc[sell, 'ATR_Signal'] = -1

    print("ATR Breakout - Buy Signals:", buy.sum(), "Sell Signals:", sell.sum())
    return df


#Q3

In [13]:
def backtest_engine(df, signal_column='Final_Signal', initial_capital=1000, max_short_days=5):
    capital = initial_capital
    position = None
    entry_price = 0
    entry_day = 0
    trades = []

    df = df.reset_index(drop=True)

    for i in range(len(df)):
        price = df.loc[i, 'Close']
        signal = df.loc[i, signal_column]

        # === Entry Logic ===
        if position is None:
            if signal == 1:
                # Long Entry
                position = 'long'
                entry_price = price
                entry_day = i
            elif signal == -1:
                # Short Entry
                position = 'short'
                entry_price = price
                entry_day = i

        # === Exit Logic ===
        elif position == 'long':
            if signal == -1 or i == len(df) - 1:
                # Close Long
                exit_price = price
                pnl = (exit_price / entry_price) * capital
                trades.append(('long', entry_day, i, entry_price, exit_price, pnl))
                capital = pnl
                position = None

        elif position == 'short':
            if signal == 1 or (i - entry_day >= max_short_days) or i == len(df) - 1:
                # Close Short
                exit_price = price
                pnl = (entry_price / exit_price) * capital
                trades.append(('short', entry_day, i, entry_price, exit_price, pnl))
                capital = pnl
                position = None

    # Print summary
    print(f"Final Capital: ₹{capital:.2f}")
    print(f"Total Trades: {len(trades)}")
    long_trades = [t for t in trades if t[0] == 'long']
    short_trades = [t for t in trades if t[0] == 'short']
    print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")

    return trades, capital
